# Get all the data from users on Admin

## Scraping on a site with login

In [1]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./scraper_ETL/extract/config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config


In [2]:
# Get the url on the config file
host = config()['website_data']['url']
# host

In [3]:
#Import selenium
from selenium import webdriver

# Setup driver
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./driver/chromedriver', options=options)

# Get the site
driver.get(host)


In [4]:
# Setup login
# Find the login elements and ensure that there are only one of each type

driver.find_elements_by_xpath('//input[@name="usuario"]')


[<selenium.webdriver.remote.webelement.WebElement (session="36f3bc761351bd1c613f91c4f5b58ec6", element="04de043f-727f-4f91-913d-6dfcf109d230")>]

In [5]:
driver.find_elements_by_xpath('//input[@name="pwd"]')

[<selenium.webdriver.remote.webelement.WebElement (session="36f3bc761351bd1c613f91c4f5b58ec6", element="d83f2e2a-116f-4754-8a5f-8ce2df01aba9")>]

In [7]:
# Get the credential data
user = config()['website_data']['user']
password = config()['website_data']['password']

alt_bntAceptarLogin = config()['website_data']['labels']['page01']['alt_bntAceptarLogin']


In [8]:
# Login
driver.find_element_by_xpath('//input[@name="usuario"]').send_keys(user)
driver.find_element_by_xpath('//input[@name="pwd"]').send_keys(password)

# Click to login... the element is an input without name and an alt tag defined on config.yaml: 
    # <input type="image" name src="bt_aceptar.gif" alt="Entar al Sistema">
driver.find_element_by_xpath('//input[@alt="' + alt_bntAceptarLogin + '"]').click()

### Successful login!

## Navigate to the important data

In [9]:
# Locate the menu option
# <a href="javascript:catchSelection(4000);">Control de Usuarios</a>
text_menuCtrlUsuarios = config()['website_data']['labels']['page02']['text_menuCtrlUsuarios']

driver.find_element_by_xpath('//a[text()="' + text_menuCtrlUsuarios + '"]')

<selenium.webdriver.remote.webelement.WebElement (session="36f3bc761351bd1c613f91c4f5b58ec6", element="133eca60-fdab-46f9-bf90-2c833b5cdff2")>

In [10]:
driver.find_element_by_xpath('//a[text()="' + text_menuCtrlUsuarios + '"]').click()

In [11]:
# Now, the next element is a select menu, so we need to import Select from Selenium
from selenium.webdriver.support.select import Select


In [12]:
# Inspect the select and print the options

# First we get the name of the select menu
name_selectMenu = config()['website_data']['labels']['page03']['name_selectMenu']

test = Select(driver.find_element_by_name(name_selectMenu))
# test.select_by_value("Consulta de Usuarios")
print(test.options[0].text)
print(test.options[1].text)
print(test.options[2].text)
print(test.options[3].text)
print(test.options[4].text)

Seleccione
Alta de Usuarios
Baja de Usuarios
Modificar Usuarios
Consulta de Usuarios


In [15]:
# Select the correct option
test = Select(driver.find_element_by_name(name_selectMenu))
option_selectMenu = config()['website_data']['labels']['page03']['option_selectMenu']
test.select_by_value(option_selectMenu)

In [16]:
# Now, we had selected the option on the menu, click on button "Aceptar"
alt_bntAceptarSelect = config()['website_data']['labels']['page03']['alt_bntAceptarSelect']
driver.find_element_by_xpath('//img[@alt="' + alt_bntAceptarSelect + '"]').click()

In [17]:
# Query all the accounts

# <img src="bt_consulta.gif" alt="Consultar" ... >
# Click on button
alt_bntConsulta = config()['website_data']['labels']['page04']['alt_bntConsulta']
driver.find_element_by_xpath('//img[@alt="' + alt_bntConsulta + '"]').click()

## Get the data

In [19]:
# Inspect pagination

# Find element
# rows = driver.find_element_by_xpath('//table[@class="txt"]').get_attribute('cellspacing')
# rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr') //26 rows (max 13 rows)
# Select rows of data without the titles row (not @valign) and even rows // 65 data
even_rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txtb"]')

len(even_rows)

65

In [20]:
# Select rows of data without the titles row (not @valign) and even rows // 60 data (max 12 rows)
odd_rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txt"]')

len(odd_rows)


60

In [ ]:
# Testing with the first and second rows
i = 0
j = 0

curp = even_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = even_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = even_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = even_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = even_rows[i].text
print('Grupo:', grupo)
i = i + 1

curp = odd_rows[j].text
print('CURP:', curp)
j = j + 1
matricula = odd_rows[j].text
print('Matrícula:', matricula)
j = j + 1
nombre = odd_rows[j].text
print('Nombre:', nombre)
j = j + 1
cuenta = odd_rows[j].text
print('Cuenta:', cuenta)
j = j + 1
grupo = odd_rows[j].text
print('Grupo:', grupo)
j = j + 1

In [22]:
# Or, even better, select all the data in the same object with:

all_rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "txt")]')

len(all_rows)


125

In [ ]:
# Testing with the first and second rows
i = 0

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1
print("")

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1


In [ ]:
# Testing with another row
i=120

print("Last Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1

### Successful scrapper on page data!